In [1]:
import dask.dataframe as ddf
import pandas as pd
import os

In [2]:
EOD_2024=ddf.read_csv('Matriz_od_modo_censo_2024.csv')

In [ ]:
cod_terr = pd.read_csv('cod_territoriales_especificos.csv', encoding='utf-8')
cod_terr.columns = ['codigo', 'nombre']
EOD_2024_pd = EOD_2024.compute() if hasattr(EOD_2024, 'compute') else EOD_2024

# Mapear comuna_origen
EOD_2024_pd = EOD_2024_pd.merge(
    cod_terr.rename(columns={'codigo': 'comuna_origen', 'nombre': 'nombre_origen'}),
    on='comuna_origen',
    how='left'
)

# Mapear comuna_destino
EOD_2024_pd = EOD_2024_pd.merge(
    cod_terr.rename(columns={'codigo': 'comuna_destino', 'nombre': 'nombre_destino'}),
    on='comuna_destino',
    how='left'
)

EOD_2024 = ddf.from_pandas(EOD_2024_pd, npartitions=4) if 'ddf' in str(type(EOD_2024)) else EOD_2024_pd

In [4]:
EOD_2024.head(10)

,comuna_origen,comuna_destino,sexo,grupo_edad,medio_transp,n_personas,nombre_origen,nombre_destino
0,5802,998,1,50,2,31,Limache,Varias comunas o países
1,5802,5802,2,45,3,119,Limache,Limache
2,4303,4303,1,55,2,73,Monte Patria,Monte Patria
3,1101,1101,1,70,1,282,Iquique,Iquique
4,1101,1101,1,45,2,871,Iquique,Iquique
5,13124,13128,2,45,7,2,Pudahuel,Renca
6,13124,13128,1,45,7,5,Pudahuel,Renca
7,8111,8111,1,30,3,93,Tomé,Tomé
8,8111,8111,2,30,1,211,Tomé,Tomé
9,13101,13130,2,30,2,228,Santiago,San Miguel


In [5]:
EOD_2024 = EOD_2024.drop(columns=['comuna_origen', 'comuna_destino'])

In [6]:
comunas_rm = [
    "Cerrillos",
    "Cerro Navia",
    "Conchalí",
    "El Bosque",
    "Estación Central",
    "Huechuraba",
    "Independencia",
    "La Cisterna",
    "La Florida",
    "La Granja",
    "La Pintana",
    "La Reina",
    "Las Condes",
    "Lo Barnechea",
    "Lo Espejo",
    "Lo Prado",
    "Macul",
    "Maipú",
    "Ñuñoa",
    "Pedro Aguirre Cerda",
    "Peñalolén",
    "Providencia",
    "Pudahuel",
    "Quilicura",
    "Quinta Normal",
    "Recoleta",
    "Renca",
    "San Joaquín",
    "San Miguel",
    "San Ramón",
    "Vitacura",
    "Puente Alto",
    "Pirque",
    "San José de Maipo",
    "Colina",
    "Lampa",
    "Tiltil",
    "San Bernardo",
    "Buin",
    "Calera de Tango",
    "Paine",
    "Melipilla",
    "Alhué",
    "Curacaví",
    "María Pinto",
    "San Pedro",
    "Talagante",
    "El Monte",
    "Isla de Maipo",
    "Padre Hurtado",
    "Peñaflor",
    "Santiago"
]


In [7]:
EOD_2024 = EOD_2024[EOD_2024['nombre_destino'].isin(comunas_rm)]

In [ ]:
folder = 'EOD_STGO'
viajes = pd.read_csv(os.path.join(folder, 'viajes.csv'), sep=';', encoding='utf-8')
personas = pd.read_csv(os.path.join(folder, 'personas.csv'), sep=';', encoding='utf-8')
edades = pd.read_csv(os.path.join(folder, 'Edadpersonas.csv'), sep=';', encoding='utf-8')
difusion = pd.read_csv(os.path.join(folder, 'ViajesDifusion.csv'), sep=';', encoding='utf-8')
comunas = pd.read_csv('EOD_STGO/Tablas_parametros/Comunas.csv', sep=',', encoding='utf-8')
comunas.columns = ['Id', 'Comuna']



,sexo,difusion,proposito,proposito_agregado,rango_edad,nombre_comuna_origen,nombre_comuna_destino
0,2,2,7.0,3.0,40,Maipú,Maipú
1,1,2,1.0,1.0,20,Maipú,Las Condes
2,1,2,7.0,1.0,20,Las Condes,Maipú
3,2,2,1.0,1.0,45,Maipú,Ñuñoa
4,2,2,7.0,1.0,45,Ñuñoa,Maipú


In [ ]:
df = viajes[['Hogar', 'Persona', 'Viaje', 'ComunaOrigen', 'ComunaDestino', 'Proposito', 'PropositoAgregado']]
df = df.merge(personas[['Hogar', 'Persona', 'Sexo']], on=['Hogar', 'Persona'], how='left')
df = df.merge(edades, on='Persona', how='left')
df = df.merge(difusion, on='Viaje', how='left')

In [ ]:
resultado = df[['ComunaOrigen', 'ComunaDestino', 'Sexo', 'Edad', 'ModoDifusion', 'Proposito', 'PropositoAgregado']].copy()
resultado.columns = ['comuna_origen', 'comuna_destino', 'sexo', 'edad', 'difusion', 'proposito', 'proposito_agregado']


In [ ]:
bins = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, float('inf')]
labels = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]
resultado['rango_edad'] = pd.cut(resultado['edad'], bins=bins, labels=labels, right=False)

In [ ]:
resultado = resultado.merge(
    comunas.rename(columns={'Id': 'comuna_origen', 'Comuna': 'nombre_comuna_origen'}),
    on='comuna_origen',
    how='left'
)

resultado = resultado.merge(
    comunas.rename(columns={'Id': 'comuna_destino', 'Comuna': 'nombre_comuna_destino'}),
    on='comuna_destino',
    how='left'
)
resultado.drop(columns=['comuna_origen', 'comuna_destino','edad'], inplace=True)
resultado.head()

In [11]:
resultado = resultado[(resultado['proposito'] == 1.0) | (resultado['proposito'] == 2.0)& (resultado['proposito_agregado']==1.0) ]

In [12]:
resultado.head(5)

,sexo,difusion,proposito,proposito_agregado,rango_edad,nombre_comuna_origen,nombre_comuna_destino
1,1,2,1.0,1.0,20,Maipú,Las Condes
3,2,2,1.0,1.0,45,Maipú,Ñuñoa
12,2,1,1.0,1.0,30,Maipú,Maipú
23,1,7,1.0,1.0,35,Maipú,Pudahuel
27,2,4,1.0,1.0,35,Maipú,Santiago


In [13]:
resultado=resultado.drop(columns=['proposito_agregado','proposito'])

In [ ]:
columnas = ['sexo', 'rango_edad', 'difusion', 'nombre_comuna_origen', 'nombre_comuna_destino']

resultado_agrupado = (resultado[columnas]
                      .value_counts()
                      .reset_index(name='n_personas'))

In [18]:
resultado_agrupado.head(10)

,sexo,rango_edad,difusion,nombre_comuna_origen,nombre_comuna_destino,n_personas
0,1,30,8,Santiago,Santiago,34
1,1,25,8,Santiago,Santiago,26
2,1,45,8,Santiago,Santiago,25
3,2,30,8,Santiago,Santiago,25
4,1,35,8,Santiago,Santiago,25
5,2,40,8,Santiago,Santiago,21
6,1,50,8,Santiago,Santiago,20
7,2,20,2,Maipú,Santiago,19
8,1,35,1,Las Condes,Las Condes,19
9,1,30,2,Maipú,Santiago,18


In [19]:
EOD_2024=EOD_2024[['sexo', 'grupo_edad', 'medio_transp', 'nombre_origen', 'nombre_destino', 'n_personas']]

In [20]:
EOD_2024.head(10)

,sexo,grupo_edad,medio_transp,nombre_origen,nombre_destino,n_personas
5,2,45,7,Pudahuel,Renca,2
6,1,45,7,Pudahuel,Renca,5
9,2,30,2,Santiago,San Miguel,228
10,2,40,2,Isla de Maipo,Talagante,32
11,1,55,2,Pudahuel,Maipú,72
18,2,45,2,Huechuraba,Huechuraba,158
19,1,20,2,Huechuraba,Las Condes,68
20,2,40,2,Maipú,Quilicura,65
21,2,25,2,Quinta Normal,Providencia,306
24,1,75,2,Providencia,Santiago,37
